In [1]:
import pandas as pd
import os
import numpy as np

In [3]:
root = os.path.join("..","Data")
df = pd.read_excel(os.path.join(root, "DEMAND_FILE_PHASE2.xlsx"))
df = df.iloc[0:2].T.reset_index(drop=True)
df.columns = ['Date','Sales']
df.drop(0, inplace=True)
df['Sales'] = df['Sales'].astype('float')
df['Date'] = pd.to_datetime(df['Date'])

train_df = df[df['Date']<"2024-01-01"].reset_index(drop=True)
test_df = df[df['Date'] >= "2024-01-01"].reset_index(drop=True)

In [36]:
def create_sliding_window_sets(df, window_size=30):
    sales = df['Sales'].values
    dates = df['Date'].values
    X, Y, X_dates = [], [], []

    for i in range(window_size, len(sales) - window_size):
        x_window = sales[i - window_size:i]         
        y_window = sales[i + 1:i + 1 + window_size].sum() 
        today_date = dates[i]                       

        X.append(x_window)
        Y.append(y_window)
        X_dates.append(today_date)

    return np.array(X), np.array(Y), np.array(X_dates)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

num_days = 15
X_train, Y_train, dates_train = create_sliding_window_sets(train_df,num_days)
X_test, Y_test, dates_test = create_sliding_window_sets(test_df,num_days)

def mape(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Avoid division by zero by adding a small epsilon where y_true is zero
    epsilon = tf.keras.backend.epsilon()
    y_true = tf.where(tf.equal(y_true, 0), epsilon, y_true)

    return tf.reduce_mean(tf.abs((y_true - y_pred) / y_true)) * 100

# Build FFNN model
model = models.Sequential([
    layers.Input(shape=(num_days,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1) 
])
early_stopping = EarlyStopping(
monitor='mape',
mode = 'min',
patience=15,
restore_best_weights=True )

model.compile(optimizer='adam', loss='mse', metrics=[mape])

# Train the model
history = model.fit(X_train, Y_train, epochs=50, batch_size=4, verbose=1,callbacks=[early_stopping])


Epoch 1/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1848423350272.0000 - mape: 30.7500
Epoch 2/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 351829327872.0000 - mape: 22.6765
Epoch 3/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 384078839808.0000 - mape: 23.1816
Epoch 4/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 441269485568.0000 - mape: 23.6231
Epoch 5/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 392258158592.0000 - mape: 23.3626
Epoch 6/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 428443992064.0000 - mape: 25.1129
Epoch 7/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 368673423360.0000 - mape: 24.2360
Epoch 8/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 383618351104.0000 - mape: 23.2661
Epoch 9/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 422915538944.0000 - mape: 23.6260
Epoch 10/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 390475841536.0000 - mape: 23.1942
Epoch 11/50
334/334 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - los

In [35]:
Y_pred = model.predict(X_test, verbose=0)
print(f"The MAPE is: {mape(Y_test,Y_pred)}")

The MAPE is: 12.375097274780273


In [31]:
import plotly.graph_objects as go

dates = pd.date_range(start=dates_test[0], periods=len(X_test))
pred_df = pd.DataFrame({'Date':dates,'Actual Sales':Y_test, 'Predicted Sales': Y_pred.reshape(-1)})
pred_df['Error'] = (pred_df['Actual Sales'] - pred_df['Predicted Sales'])*100/pred_df['Actual Sales']
fig = go.Figure()
fig.add_trace(go.Line(x = pred_df['Date'],
                    y = pred_df['Actual Sales'],
                    mode = 'lines+markers',
                    name = 'Actual Sales'))
fig.add_trace(go.Line(x = pred_df['Date'],
                    y = pred_df['Predicted Sales'],
                    mode = 'lines+markers',
                    name = 'Predicted Sales'))
fig.update_layout(xaxis_title = 'Date', yaxis_title = 'Sum Sales', title = f"Predicted vs Actual Sum Sales. MAPE={mape(Y_test,Y_pred)}")
fig.show()

fig = go.Figure()
fig.add_trace(go.Line(x = pred_df['Date'],
                    y = pred_df['Error'],
                    mode = 'lines+markers',
                    name = 'Error'))
fig.update_layout(xaxis_title = 'Date', yaxis_title = "%Error", title = f"%Error")
fig.show()

c:\Users\milan\Desktop\CDIS Project\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


